In [1]:
import sys
import json
sys.path.append('..')
from src.common import *
from src.analysis.model_performances import *
from copy import deepcopy
from helpers import *
import pandas as pd


TO_PRETTY |= {(WITHOUT_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Baseline', 
              (WITHOUT_RANDOM_SUB, WITH_RAMIFICATIONS): 'Baseline + R.',
             (WITH_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Obfus. Baseline',
             (WITH_RANDOM_SUB, WITH_RAMIFICATIONS): 'Obfus. Baseline + R.',
}

In [6]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = ACCURACY_SCORE_KEY #F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = score_key)

ids_file_name = 'dataset_ids.test.pruned'  # None
save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'
stats_all = collect_stats_all(tf_answer_type(score_key = score_key), save_main_dir=save_main_dir)
print(len(stats_all))
plan_length= 19

save_dir = os.path.join(save_main_dir, 'tables', 'by_categories')
os.makedirs(save_dir, exist_ok=True)


100%|██████████| 27648/27648 [00:04<00:00, 6765.32it/s]

16721


In [7]:
def to_df_by_category(results_all, answer_type,  
                      model_name,
                      prompt_type='few_shot_5',
                      domain = ALL_DOMAINS_KEY,
                      plan_length=19):

    index = []
    data = []    
    for question_category in QUESTION_CATEGORIES+[ALL_QUESTION_CATEGORIES_KEY]:
        index.append(TO_PRETTY[question_category])
        data_columns = {}
        for subs in [WITHOUT_RANDOM_SUB, WITH_RANDOM_SUB]:
            for ramifications in [WITHOUT_RAMIFICATIONS, WITH_RAMIFICATIONS]:
                res_obj = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
                if res_obj:
                    mean = res_obj['result']
                    sem = None
                    if res_obj['result_other']:
                        sem = res_obj['result_other']['sem']
                    not_corrupted = res_obj['stats']['num_not_corrupted']
                    final_res = (mean, sem, not_corrupted)
                else:
                    final_res = (None, None, None)
                final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
                final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
                data_columns[TO_PRETTY[(subs, ramifications)]] = final_res
        data.append(data_columns)
    return pd.DataFrame(data, index = index)

# Subs and Ramfications

In [9]:
plan_length = 19
model_name = 'gemini'
prompt_type = 'few_shot_1'
df = to_df_by_category(stats_all, answer_type,model_name=model_name, prompt_type=prompt_type, plan_length=plan_length)
df
        
caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
save_key = f'{model_name}.{prompt_type}.{plan_length}'

latex_table_all = to_latex_table(df, caption_nl, label=save_key)
with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
    f.write(latex_table_all)

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


# Fluent Types 